In [1]:
import torch
from PopSynthesis.Methods.GNN_activity.model import TravelGNN, train_model

In [2]:
# Load the starting and expected graphs
starting_graph_pyg = torch.load("data/start_graph.pt")
expected_graph_pyg = torch.load("data/expected_graph.pt")

In [3]:
model = TravelGNN(hidden_channels=64)
optimizer = torch.optim.Adam(model.parameters(), lr=0.005)

criterion_edge = torch.nn.BCEWithLogitsLoss()
criterion_duration = torch.nn.MSELoss()
criterion_joint = torch.nn.BCEWithLogitsLoss()

data = expected_graph_pyg.to('cpu')

In [4]:
model = train_model(model, data)

Epoch 0: Total Loss=402.1098, Edge=1.2738, Duration=394.9019, Joint=5.9342
Epoch 5: Total Loss=16.3306, Edge=0.8216, Duration=14.5203, Joint=0.9887
Epoch 10: Total Loss=39.0285, Edge=0.7106, Duration=36.8875, Joint=1.4304
Epoch 15: Total Loss=14.6816, Edge=0.6647, Duration=12.9781, Joint=1.0388
Epoch 20: Total Loss=5.4867, Edge=0.7023, Duration=4.0806, Joint=0.7037
Epoch 25: Total Loss=5.0713, Edge=0.6346, Duration=3.6678, Joint=0.7689
Epoch 30: Total Loss=4.3587, Edge=0.5717, Duration=3.0931, Joint=0.6939
Epoch 35: Total Loss=2.6123, Edge=0.5548, Duration=1.4164, Joint=0.6411
Epoch 40: Total Loss=1.8088, Edge=0.5781, Duration=0.5930, Joint=0.6376
Epoch 45: Total Loss=1.7373, Edge=0.6185, Duration=0.5189, Joint=0.5999
Epoch 49: Total Loss=1.6253, Edge=0.6407, Duration=0.4014, Joint=0.5832


In [5]:
def infer(model, data, max_duration=1440): # just to test
    model.eval()
    with torch.no_grad():
        embeddings = model(data.x_dict, data.edge_index_dict)
        person_emb = embeddings["person"]
        purpose_emb = embeddings["purpose"]

        edge_index = data["person", "performs", "purpose"].edge_index
        pos_person_emb = person_emb[edge_index[0]]
        pos_purpose_emb = purpose_emb[edge_index[1]]

        edge_probs = torch.sigmoid(model.predict_edges(pos_person_emb, pos_purpose_emb))
        predicted_durations = model.predict_duration(pos_person_emb, pos_purpose_emb) * max_duration
        joint_probs = torch.sigmoid(model.predict_joint(pos_person_emb, pos_purpose_emb))

    return edge_probs, predicted_durations, joint_probs


In [6]:
# Run inference
edge_probs, durations, joint_probs = infer(model, data)

print("Edge Probabilities (first 5):", edge_probs[:5])
print("Predicted Durations (min, first 5):", durations[:5])
print("Joint Activity Probabilities (first 5):", joint_probs[:5])

Edge Probabilities (first 5): tensor([0.6273, 0.6126, 0.6298, 0.6273, 0.7878])
Predicted Durations (min, first 5): tensor([ -561.4427,    38.1671,  -576.3266,  -561.4427, -1892.6478])
Joint Activity Probabilities (first 5): tensor([0.4350, 0.4587, 0.4341, 0.4350, 0.9190])


In [7]:
# save model
torch.save({
    "model_state_dict": model.state_dict(),
    "hidden_channels": 64,  # Any hyperparameters you may have
}, "data/trained_travel_gnn_model.pt")